In [5]:
# Updated 2024.11.02
import pandas as pd
import requests
import csv
from datetime import datetime, timedelta

# Your API Key from The Odds API
API_KEY = 'Insert API Key Here'

# Define constants for the API call
SPORT = 'mma_mixed_martial_arts'  # MMA sport key
REGIONS = 'us'  # Focus on US market
MARKETS = 'h2h'  # Only fetch head-to-head markets
ODDS_FORMAT = 'american'  # American odds format
DATE_FORMAT = 'iso'  # ISO date format for timestamps

# Base URL for the Odds API
BASE_URL = 'https://api.the-odds-api.com/v4/historical/sports'

def get_event_dates_between(masterlist_df, start_date, end_date):
    """
    Filters an event_masterlist (generated by UFCStats Scraper) for dates within a specified range and applies a 12-hour offset.
    
    Parameters:
    - masterlist_df (DataFrame): A pandas DataFrame containing a list of events, including an 'event_date' column with event dates.
    - start_date (str or datetime-like): The start date to filter events. Events on or after this date are included in the output.
    - end_date (str or datetime-like): The end date to filter events. Events on or before this date are included in the output.

    Process:
    1. Converts the `start_date` and `end_date` inputs to datetime objects, allowing for flexible input formats (string or datetime).
    2. Filters `masterlist_df` to include only rows with event dates within the specified range from `start_date` to `end_date`.
    3. Applies a 12-hour offset to the event dates in the filtered DataFrame, which shifts each event date by 12 hours.
       This adjustment can be useful for accounting for timezone differences or standardizing event times.
    4. Extracts unique event dates from the adjusted 'event_date' column.
    
    Returns:
    - event_dates (list): A list of unique event dates (with the 12-hour offset applied) within the specified date range.
    - num_events (int): The count of unique events found within the range, useful for quick reference or logging.
    
    Example:
    --------
    >>> masterlist_df = pd.DataFrame({'event_date': ['2024-08-31', '2024-08-24', '2024-08-17', '2024-08-10']})
    >>> start_date = '2024-08-01'
    >>> end_date = '2024-08-30'
    >>> get_event_dates_between(masterlist_df, start_date, end_date)
    ([Timestamp('2024-08-10 12:00:00'), Timestamp('2024-08-17 12:00:00'), Timestamp('2023-08-24 12:00:00')], 3)

    Note:
    - This function is intended for event scheduling contexts where all events are expected to have distinct dates,
      and the 12-hour offset is applied uniformly to all events within the specified date range.
    - The function assumes that `masterlist_df` includes a column named 'event_date' containing datetime-compatible data.
    """
    # Convert input dates to datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter masterlist for event dates within the given range
    filtered_masterlist = masterlist_df[(masterlist_df['event_date'] >= start_date) & (masterlist_df['event_date'] <= end_date)].copy()
    
    # Apply a 12-hour offset to the event dates using .loc[] to avoid the warning
    filtered_masterlist.loc[:, 'event_date'] = filtered_masterlist['event_date'] + pd.Timedelta(hours=12)
    
    # Extract unique event dates
    event_dates = filtered_masterlist['event_date'].unique()
    
    # Convert event dates back to a list and return
    return list(event_dates), len(event_dates)


def get_historical_mma_odds(api_key, snapshot_date):
    """
    Retrieves historical MMA odds for a specified timestamp from The Odds API.

    Parameters:
    - api_key (str): The API key for authenticating requests to The Odds API.
    - snapshot_date (str): The date and time for which to retrieve odds, formatted as an ISO 8601 timestamp 
                           (e.g., '2024-08-24T12:00:00Z').

    Process:
    1. Constructs the API endpoint URL for fetching MMA odds using a base URL and sport-specific path.
    2. Prepares query parameters, including:
       - `api_key`: The API key provided by the user for authentication.
       - `regions`: Specifies the market region, e.g., 'us'.
       - `markets`: Type of betting market to retrieve, e.g., 'h2h' (head-to-head).
       - `date`: The target timestamp (`snapshot_date`) in ISO format for the requested historical odds.
       - `oddsFormat`: Specifies the format of the odds, such as 'american'.
       - `dateFormat`: Specifies the format of the timestamp in the response.
    3. Sends an HTTP GET request to The Odds API using the constructed URL and parameters.

    Returns:
    - odds_data (dict): Parsed JSON response containing the odds data for the specified timestamp, 
                        or `None` if the request fails.
    - response (Response): The full response object from the API request, allowing access to status 
                           codes and headers.

    Example:
    --------
    >>> api_key = 'YOUR_API_KEY'
    >>> snapshot_date = '2024-08-24T12:00:00Z'
    >>> odds_data, response = get_historical_mma_odds(api_key, snapshot_date)
    >>> if odds_data:
    ...     print(f"Odds retrieved for timestamp: {snapshot_date}")
    ... else:
    ...     print(f"Failed to retrieve odds for timestamp: {snapshot_date}")
    Odds retrieved for timestamp: 2024-08-24T12:00:00Z

    Note:
    - Ensure that the `api_key` is valid and active to avoid authentication errors.
    - The `snapshot_date` should be provided in ISO 8601 format to match The Odds API's expected format.
    - Use the `response` object to handle HTTP status codes or rate limit headers for additional context.
    """
    url = f'{BASE_URL}/{SPORT}/odds'
    params = {
        'api_key': api_key,
        'regions': REGIONS,
        'markets': MARKETS,
        'date': snapshot_date,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f'Failed to get historical odds: status_code {response.status_code}, response body {response.text}')
        return None, response

    # Return the parsed JSON response and the response object to access headers
    return response.json(), response


def save_odds_to_csv(odds_data, file_name):
    """
    Saves structured odds data to a CSV file with UTF-8 encoding to handle special characters.
    
    Parameters:
    - odds_data (dict): The odds data to save, structured with details for each event, bookmaker, market, and outcome.
    - file_name (str): The name of the CSV file to save the data to. The file is appended to if it already exists.

    Process:
    1. Opens (or creates if it doesn’t exist) the specified CSV file in append mode with UTF-8 encoding.
       UTF-8 encoding ensures compatibility with any special characters in bookmaker or team names.
    2. Initializes a CSV writer to format and write rows to the file.
    3. Checks if the file is empty:
       - If it is, writes a header row with the column names, describing each data field in subsequent rows.
    4. Iterates through the `odds_data` dictionary to extract relevant details:
       - For each event: extracts `home_team`, `away_team`, `commence_time`.
       - For each bookmaker: extracts `title`.
       - For each market: extracts `key`.
       - For each outcome: extracts `name` and `price`.
    5. Writes each outcome as a row in the CSV file, where each row contains:
       `[timestamp, home_team, away_team, commence_time, bookmaker, market, outcome_name, odds_price]`.

    Example:
    --------
    >>> odds_data = {
    ...     'timestamp': '2024-08-24T12:00:00Z',
    ...     'data': [
    ...         {
    ...             'home_team': 'Fighter A',
    ...             'away_team': 'Fighter B',
    ...             'commence_time': '2024-08-24T12:00:00Z',
    ...             'bookmakers': [
    ...                 {
    ...                     'title': 'Bookmaker1',
    ...                     'markets': [
    ...                         {
    ...                             'key': 'h2h',
    ...                             'outcomes': [
    ...                                 {'name': 'Fighter A', 'price': -150},
    ...                                 {'name': 'Fighter B', 'price': +130}
    ...                             ]
    ...                         }
    ...                     ]
    ...                 }
    ...             ]
    ...         }
    ...     ]
    ... }
    >>> file_name = 'mma_odds.csv'
    >>> save_odds_to_csv(odds_data, file_name)

    Note:
    - This function appends data to `file_name`, so if used repeatedly, it will continue to add rows without overwriting.
    - It is designed to handle nested dictionaries as structured by The Odds API, assuming data includes bookmakers, markets, and outcomes.
    - A header row is written only if the file is empty, ensuring consistency in CSV structure.
    """
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:  # Specify utf-8 encoding
        writer = csv.writer(file)
        
        # Write header if the file is empty
        if file.tell() == 0:
            writer.writerow(['Timestamp', 'Home Team', 'Away Team', 'Commence Time', 'Bookmaker', 'Market', 'Outcome Name', 'Odds Price'])
        
        # Write odds data to the CSV file
        for event in odds_data['data']:
            for bookmaker in event['bookmakers']:
                for market in bookmaker['markets']:
                    for outcome in market['outcomes']:
                        writer.writerow([odds_data['timestamp'], 
                                         event['home_team'], 
                                         event['away_team'], 
                                         event['commence_time'], 
                                         bookmaker['title'], 
                                         market['key'], 
                                         outcome['name'], 
                                         outcome['price']])


def get_and_save_odds_for_dates(api_key, event_dates, start_date, end_date, limit=None):
    """
    Iterates through a list of event dates, retrieves historical odds for each date, 
    and saves the data to a CSV file named according to the date range.

    Parameters:
    - api_key (str): The Odds API key for authenticating requests.
    - event_dates (list of datetime): A list of event dates, each with a 12-hour offset applied.
    - start_date (str): The start date for the event range, used for naming the output file.
    - end_date (str): The end date for the event range, used for naming the output file.
    - limit (int, optional): The maximum number of event dates to process. 
                             If None, all dates in `event_dates` are processed.

    Process:
    1. Generates a dynamic file name using the specified date range:
       `mma_odds_(end_date to start_date).csv`.
       This file will contain all odds data retrieved within the specified date range.
    2. If `limit` is specified, truncates `event_dates` to the first `limit` items.
    3. Iterates through the list of `event_dates`:
       - Formats each event date into an ISO 8601 timestamp (e.g., '2024-08-24T12:00:00Z').
       - Calls `get_historical_mma_odds` to retrieve odds data for the specific timestamp.
       - If data is retrieved, saves it to the CSV file using `save_odds_to_csv`.
       - Logs each action, including whether data was saved and the API’s request status headers.

    Example:
    --------
    >>> api_key = 'YOUR_API_KEY'
    >>> event_dates = [pd.Timestamp('2024-08-24 12:00:00'), pd.Timestamp('2024-08-17 12:00:00')]
    >>> start_date = '2024-08-01'
    >>> end_date = '2024-08-24'
    >>> get_and_save_odds_for_dates(api_key, event_dates, start_date, end_date, limit=1)

    Expected Output:
    Fetching odds for timestamp: 2024-08-24T12:00:00Z
    Odds saved for timestamp: 2024-08-24T12:00:00Z
    Remaining requests: [number of remaining requests from headers]
    Used requests: [number of used requests from headers]

    Note:
    - The file name is dynamically generated and includes the start and end dates.
    - This function will print the number of remaining and used API requests, based on the response headers.
    - To avoid excessive API usage, specify a `limit` if testing with multiple dates.
    """
    # Generate a dynamic file name based on the date range
    file_name = f'mma_odds_({end_date} to {start_date}).csv'
    
    if limit:
        event_dates = event_dates[:limit]  # Apply limit if specified

    for i, event_date in enumerate(event_dates):  
        snapshot_date = event_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        print(f"\nFetching odds for timestamp: {snapshot_date}")
        odds_data, response = get_historical_mma_odds(api_key, snapshot_date)

        if odds_data:
            save_odds_to_csv(odds_data, file_name)
            print(f"Odds saved for timestamp: {snapshot_date}")
        else:
            print(f"No data available for timestamp {snapshot_date}")

        print(f"\nRemaining requests: {response.headers.get('x-requests-remaining', 'N/A')}")
        print(f"Used requests: {response.headers.get('x-requests-used', 'N/A')}")

        
def main():
    # Load masterlist CSV with date parsing
    masterlist_file = "cleaned_event_masterlist.csv" # Generated using UFCStats Scraper (different repository)
    masterlist_df = pd.read_csv(masterlist_file, encoding='utf-8', parse_dates=['event_date'])

    # Define date range for filtering
    start_date = '2020-06-06' # First day TheOddsAPI began archiving UFC h2h odds
    end_date = '2024-08-24'

    # Get event dates within the range
    event_dates_list, num_events = get_event_dates_between(masterlist_df, start_date, end_date)
    print(f"There are {num_events} events between {start_date} and {end_date}.")
    print(f"Event dates (with 12-hour offset): {event_dates_list}")

    # Generate a dynamic file name based on the date range
    file_name = f'mma_odds_({start_date} to {end_date}).csv'

    # Retrieve and save odds data for the first 5 event dates
    get_and_save_odds_for_dates(API_KEY, event_dates_list, start_date, end_date, limit=5) 

if __name__ == "__main__":
    main()


There are 186 events between 2020-06-06 and 2024-08-24.

Event dates (with 12-hour offset): [Timestamp('2024-08-24 12:00:00'), Timestamp('2024-08-17 12:00:00'), Timestamp('2024-08-10 12:00:00'), Timestamp('2024-08-03 12:00:00'), Timestamp('2024-07-27 12:00:00'), Timestamp('2024-07-20 12:00:00'), Timestamp('2024-07-13 12:00:00'), Timestamp('2024-06-29 12:00:00'), Timestamp('2024-06-22 12:00:00'), Timestamp('2024-06-15 12:00:00'), Timestamp('2024-06-08 12:00:00'), Timestamp('2024-06-01 12:00:00'), Timestamp('2024-05-18 12:00:00'), Timestamp('2024-05-11 12:00:00'), Timestamp('2024-05-04 12:00:00'), Timestamp('2024-04-27 12:00:00'), Timestamp('2024-04-13 12:00:00'), Timestamp('2024-04-06 12:00:00'), Timestamp('2024-03-30 12:00:00'), Timestamp('2024-03-23 12:00:00'), Timestamp('2024-03-16 12:00:00'), Timestamp('2024-03-09 12:00:00'), Timestamp('2024-03-02 12:00:00'), Timestamp('2024-02-24 12:00:00'), Timestamp('2024-02-17 12:00:00'), Timestamp('2024-02-10 12:00:00'), Timestamp('2024-02-03 1

In [ ]:
import csv
from collections import defaultdict
from statistics import mean
from math import copysign


def calculate_implied_probability_american_odds(odds):
    """
    Calculate the implied probability from American odds.

    American odds are typically used in the United States and differ
    from decimal odds in their representation of implied probability:
    - Positive odds (e.g., +150) indicate how much profit would be made on a $100 bet.
    - Negative odds (e.g., -200) indicate how much must be wagered to win $100.

    This function converts these American odds into implied probabilities,
    providing an estimate of the likelihood that a given outcome will occur
    according to the bookmaker.

    Parameters:
    - odds (float): The American odds value, which can be positive or negative.

    Returns:
    - float: The implied probability, rounded to 2 decimal places, representing the
             bookmaker's estimated probability of the outcome. This value ranges from 0 to 1,
             where 1 indicates 100% implied probability.

    Examples:
    - For positive American odds of +150:
        calculate_implied_probability_american_odds(150) --> 0.4 (or 40%)

    - For negative American odds of -200:
        calculate_implied_probability_american_odds(-200) --> 0.6667 (or 66.67%)

    Calculation Method:
    - For positive odds (e.g., +150): Implied probability = 100 / (odds + 100)
    - For negative odds (e.g., -200): Implied probability = abs(odds) / (abs(odds) + 100)
    """
    if odds > 0:
        return round(100 / (odds + 100), 2)  # For positive odds
    else:
        return round(abs(odds) / (abs(odds) + 100), 2)  # For negative odds


def parse_odds_data(row, fights_with_odds):
    """
    Parse a row of odds data and add it to the fights_with_odds dictionary.

    This function processes a single row from a CSV file, representing odds data for an MMA fight.
    It identifies and structures information about each fight, organizing it by the event date,
    fighters' names, and specific odds information. The data is stored in a nested dictionary,
    where each event is uniquely keyed by the combination of home team, away team, and commence time.

    Parameters:
    - row (dict): A dictionary representing a row of odds data from a CSV file. 
                  This dictionary contains details such as "Home Team", "Away Team", "Commence Time",
                  "Bookmaker", "Odds Price", and "Outcome Name".
    - fights_with_odds (dict): A dictionary that stores parsed fight data. Each entry is uniquely
                               keyed by a tuple containing the home team, away team, and event time.
                               The dictionary stores each fighter's odds and additional information
                               such as event date and bookmaker.

    Function Process:
    1. Creates a unique key (event_key) for each event using "Home Team", "Away Team", and "Commence Time".
    2. Checks if the event is already initialized in fights_with_odds. If not, it sets up the event
       by storing the event date and fighter names.
    3. Collects odds data (bookmaker and odds price) for each fighter and appends it to their respective odds list.
       The function distinguishes between the fighters using the "Outcome Name" field, which is matched
       against the "Home Team" or "Away Team".

    Example:
    Given a row with:
        {
            'Home Team': 'Fighter A',
            'Away Team': 'Fighter B',
            'Commence Time': '2024-08-24T12:00:00Z',
            'Bookmaker': 'DraftKings',
            'Odds Price': '-150',
            'Outcome Name': 'Fighter A'
        }
    This function will add this odds entry to Fighter A's odds list for the event.

    Notes:
    - "Odds Price" is stored as a float to facilitate mathematical operations.
    - The function assumes that the "Outcome Name" in the row corresponds to either the home or away fighter.

    """
    event_key = (row['Home Team'], row['Away Team'], row['Commence Time'])
    
    # Initialize event information if not already set
    if not fights_with_odds[event_key]['Event Date']:
        fights_with_odds[event_key]['Event Date'] = row['Commence Time']
        fights_with_odds[event_key]['Fighter A'] = row['Home Team']
        fights_with_odds[event_key]['Fighter B'] = row['Away Team']
    
    # Collect odds data for each fighter
    odds_info = {
        'Bookmaker': row['Bookmaker'],
        'Odds Price': float(row['Odds Price'])
    }
    if row['Outcome Name'] == fights_with_odds[event_key]['Fighter A']:
        fights_with_odds[event_key]['Fighter A Odds'].append(odds_info)
    elif row['Outcome Name'] == fights_with_odds[event_key]['Fighter B']:
        fights_with_odds[event_key]['Fighter B Odds'].append(odds_info)

        
def implied_probability_to_american_odds(probability):
    """
    Convert an implied probability to American odds.
    
    Parameters:
    - probability (float): The implied probability as a decimal (e.g., 0.55 for 55%).
    
    Returns:
    - float: The American odds, positive for underdogs and negative for favorites.
    """
    if probability >= 0.5:
        return -round(probability / (1 - probability) * 100, 2)  # Negative for favorites
    else:
        return round((1 - probability) / probability * 100, 2)  # Positive for underdogs

# Updated function to calculate average, best odds, and implied probability differences
def calculate_averages_and_best_odds(fights_with_odds):
    """
    Calculate average, best odds, and implied probability differences for each fighter.

    This function processes the odds data for each fighter in the `fights_with_odds` dictionary,
    calculating the average odds based on implied probabilities, best odds, and their associated implied probabilities.
    It also computes the implied probability difference between the average and best odds.

    Parameters:
    - fights_with_odds (dict): A nested dictionary where each event is uniquely identified by a key.
                               Each entry contains odds data for two fighters.
    """
    for fight, details in fights_with_odds.items():
        if details['Fighter A Odds']:
            # Convert each odds price to implied probability
            implied_probs_a = [calculate_implied_probability_american_odds(odds['Odds Price']) 
                               for odds in details['Fighter A Odds']]
            
            # Average the implied probabilities and convert back to American odds
            avg_implied_prob_a = mean(implied_probs_a)
            details['Fighter A Avg Odds'] = implied_probability_to_american_odds(avg_implied_prob_a)
            
            # Find the best odds and calculate implied probability difference
            details['Fighter A Best Odds'] = max(details['Fighter A Odds'], key=lambda x: x['Odds Price'])
            best_implied_prob_a = calculate_implied_probability_american_odds(details['Fighter A Best Odds']['Odds Price'])
            details['Fighter A Implied Diff'] = round((avg_implied_prob_a - best_implied_prob_a) * 100, 2)

        if details['Fighter B Odds']:
            # Convert each odds price to implied probability
            implied_probs_b = [calculate_implied_probability_american_odds(odds['Odds Price']) 
                               for odds in details['Fighter B Odds']]
            
            # Average the implied probabilities and convert back to American odds
            avg_implied_prob_b = mean(implied_probs_b)
            details['Fighter B Avg Odds'] = implied_probability_to_american_odds(avg_implied_prob_b)
            
            # Find the best odds and calculate implied probability difference
            details['Fighter B Best Odds'] = max(details['Fighter B Odds'], key=lambda x: x['Odds Price'])
            best_implied_prob_b = calculate_implied_probability_american_odds(details['Fighter B Best Odds']['Odds Price'])
            details['Fighter B Implied Diff'] = round((avg_implied_prob_b - best_implied_prob_b) * 100, 2)


def format_odds_display(fights_with_odds):
    """
    Display formatted odds and implied probability data for each fighter.

    This function prints detailed information for each event, including the
    odds data for both fighters. It provides the average odds, best odds,
    and the implied probability difference, offering insights into the value
    of odds across various bookmakers.

    Parameters:
    - fights_with_odds (dict): A nested dictionary containing data for each fight event.
                               Each entry includes:
                               - "Event Date": Date and time of the fight.
                               - "Fighter A"/"Fighter B": Names of the fighters.
                               - Odds data with bookmakers, average odds, best odds,
                                 and implied probability differences.

    Function Process:
    1. Iterates through each fight event in `fights_with_odds`.
    2. For each fighter:
       - Lists all available odds from different bookmakers.
       - Displays calculated average odds and best odds with the bookmaker name.
       - Shows the implied probability difference, which reflects the potential value
         of the odds variance across bookmakers.
    3. Prints "No odds available" if no data exists for a fighter.

    Example Output:
    For each event, this function will display:
    ```
    Event Date: 2024-08-24
    Fighter A: Fighter A Name
      Fighter A Odds:
        Bookmaker: Bookmaker Name, Odds Price: -150
      Fighter A Avg Odds: -155
      Fighter A Best Odds: -150 (Bookmaker: Bookmaker Name)
      Fighter A Implied Probability Difference: 1.5%

    Fighter B: Fighter B Name
      Fighter B Odds:
        Bookmaker: Bookmaker Name, Odds Price: +120
      Fighter B Avg Odds: +115
      Fighter B Best Odds: +120 (Bookmaker: Bookmaker Name)
      Fighter B Implied Probability Difference: 0.7%
    """
    for event_key, fight in fights_with_odds.items():
        print(f"Event Date: {fight['Event Date']}")
        print(f"Fighter A: {fight['Fighter A']}")
        
        # Fighter A odds information
        if fight['Fighter A Odds']:
            print("  Fighter A Odds:")
            for odds in fight['Fighter A Odds']:
                print(f"    Bookmaker: {odds['Bookmaker']}, Odds Price: {odds['Odds Price']}")
            print(f"  Fighter A Avg Odds: {fight['Fighter A Avg Odds']}")
            print(f"  Fighter A Best Odds: {fight['Fighter A Best Odds']['Odds Price']} (Bookmaker: {fight['Fighter A Best Odds']['Bookmaker']})")
            print(f"  Fighter A Implied Probability Difference: {fight['Fighter A Implied Diff']}%")
        else:
            print("  No odds available for Fighter A")
        
        print(f"\nFighter B: {fight['Fighter B']}")
        
        # Fighter B odds information
        if fight['Fighter B Odds']:
            print("  Fighter B Odds:")
            for odds in fight['Fighter B Odds']:
                print(f"    Bookmaker: {odds['Bookmaker']}, Odds Price: {odds['Odds Price']}")
            print(f"  Fighter B Avg Odds: {fight['Fighter B Avg Odds']}")
            print(f"  Fighter B Best Odds: {fight['Fighter B Best Odds']['Odds Price']} (Bookmaker: {fight['Fighter B Best Odds']['Bookmaker']})")
            print(f"  Fighter B Implied Probability Difference: {fight['Fighter B Implied Diff']}%")
        else:
            print("  No odds available for Fighter B")
        
        print("\n" + "=" * 50 + "\n")


def load_odds_from_csv(file_name):
    """
    Load and process odds data from a CSV file.

    This function reads odds data from a CSV file, organizes it in a structured dictionary format,
    and calculates average odds, best odds, and implied probability differences for each fighter
    in every event. The function returns a comprehensive data structure suitable for analysis and display.

    Parameters:
    - file_name (str): The file path of the CSV file containing odds data.
                       The CSV file is expected to contain fields such as "Home Team", "Away Team",
                       "Commence Time", "Bookmaker", "Odds Price", and "Outcome Name".

    Returns:
    - dict: A nested dictionary (`fights_with_odds`) with detailed odds information for each fight event.
            Each event is uniquely identified by a combination of fighters and commence time and includes:
            - "Event Date": The date and time of the event.
            - "Fighter A" and "Fighter B": Names of the two fighters.
            - "Fighter A Odds"/"Fighter B Odds": Lists of odds data from different bookmakers.
            - "Avg Odds": Average odds across bookmakers for each fighter.
            - "Best Odds": Best available odds with the bookmaker.
            - "Avg Implied"/"Best Implied": Implied probabilities of average and best odds.
            - "Implied Diff": Difference in implied probability between average and best odds, as a percentage.

    Function Process:
    1. Initializes a nested dictionary, `fights_with_odds`, to store event data for each fight.
       Each event key is a combination of "Home Team", "Away Team", and "Commence Time".
    2. Reads each row of the CSV file and processes the data using `parse_odds_data`,
       which organizes and appends the odds data for each fighter within the `fights_with_odds` structure.
    3. Calls `calculate_averages_and_best_odds` to compute the average odds, best odds, and implied probability
       differences for each fighter in each event.
    4. Returns the fully populated `fights_with_odds` dictionary, now containing detailed calculated data.

    Example:
    Given a file containing rows with data for multiple events, this function will:
    - Structure the data by event, storing odds for each fighter across bookmakers.
    - Calculate and include average odds, best odds, and implied probability metrics for each fighter.

    Notes:
    - Assumes that the CSV file is encoded in UTF-8.
    - Assumes that odds data are in American format and that the CSV file contains required fields.
    """
    fights_with_odds = defaultdict(lambda: {
        'Event Date': None,
        'Fighter A': None,
        'Fighter B': None,
        'Fighter A Odds': [],
        'Fighter B Odds': [],
        'Fighter A Avg Odds': None,
        'Fighter B Avg Implied': None,
        'Fighter A Best Implied': None,
        'Fighter A Best Odds': None,
        'Fighter A Implied Diff': None,
        'Fighter B Avg Odds': None,
        'Fighter B Avg Implied': None,
        'Fighter B Best Implied': None,
        'Fighter B Best Odds': None,
        'Fighter B Implied Diff': None
    })
    
    # Read odds data from CSV
    with open(file_name, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            parse_odds_data(row, fights_with_odds)
    
    # Calculate average, best odds, and implied probability differences for each fighter
    calculate_averages_and_best_odds(fights_with_odds)
    
    return fights_with_odds

# Main function
def main():
    # Specify the file path of the CSV file
    file_name = "Path to Odds File"
    
    # Load odds data from the CSV file
    fights_with_odds = load_odds_from_csv(file_name)
    
    # Display the loaded odds data
    format_odds_display(fights_with_odds)

# Execute the main function
if __name__ == "__main__":
    main()


Example Event (2024-08-24):

Event Date: 2024-08-24T23:00:00Z
Fighter A: Wang Cong
  Fighter A Odds:
    Bookmaker: DraftKings, Odds Price: -1200.0
    Bookmaker: BetOnline.ag, Odds Price: -1200.0
    Bookmaker: LowVig.ag, Odds Price: -1205.0
    Bookmaker: FanDuel, Odds Price: -1200.0
    Bookmaker: BetRivers, Odds Price: -1250.0
    Bookmaker: BetMGM, Odds Price: -1200.0
    Bookmaker: Bovada, Odds Price: -1400.0
    Bookmaker: BetUS, Odds Price: -1200.0
    Bookmaker: Caesars, Odds Price: -1200.0
    Bookmaker: DraftKings, Odds Price: -800.0
    Bookmaker: BetOnline.ag, Odds Price: -800.0
    Bookmaker: LowVig.ag, Odds Price: -800.0
    Bookmaker: FanDuel, Odds Price: -670.0
    Bookmaker: BetRivers, Odds Price: -670.0
    Bookmaker: BetMGM, Odds Price: -800.0
    Bookmaker: Bovada, Odds Price: -750.0
    Bookmaker: BetUS, Odds Price: -800.0
  Fighter A Avg Odds: -954.79
  Fighter A Best Odds: -670.0 (Bookmaker: FanDuel)
  Fighter A Implied Probability Difference: 3.51%

Fighter B: 